## About the Notebook (Export Sentinel Training Data)

1.) This Notebook is to export the training Sentinel Image which is created through the composition of ImageCollection. <br>
2.) You will need your export coordinates.<br>
4.) If the image is too big, it will split into pieces before being exported. The way it will be splitted depends on Google Earth Engine.

In [ ]:
from datetime import datetime

In [ ]:
# Run only once
import ee

while True:
  try:
    ee.Authenticate()
    ee.Initialize()
    break
  except Exception as e:
    print()
    print('\033[1mAuthentication failed\033[0m')
    print(e)
    print()
    print('\033[1mPlease Try Again\033[0m:')


Authentication failed
Google Earth Engine API has not been used in project 522309567947 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/earthengine.googleapis.com/overview?project=522309567947 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.

Please Try Again:



Authentication failed
Google Earth Engine API has not been used in project 522309567947 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/earthengine.googleapis.com/overview?project=522309567947 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.

Please Try Again:


KeyboardInterrupt: ignored

Get the max cloud probability.

In [ ]:
while True:
  try:
    MAX_CLOUD_PROBABILITY = int(input('Enter the max cloud probability: '))
    break
  except:
    print('Please enter a number.')
    print()

Enter the max cloud probability: 30


In [ ]:
def get_sentinel_type():
  '''
  This function gets the sentinel type.

  Returns:
  - sentinel_type: str
  '''
  while True:
    try:
      sentinel_type = int(input('Enter the sentinel type: (Sentinel 1: 1, Sentinel 2: 2) - '))
      if sentinel_type == 1:
        return 'COOPERNICUS/S2'
      elif sentinel_type == 2:
        return 'COPERNICUS/S2_SR'
      else:
        print('Please enter a valid sentinel type.')
    except:
      print('Please enter a number.')
      print()

In [ ]:
def get_file_and_folder_name():
  '''
  This function gets the file name and folder name.

  Returns:
  - file_name: str
  - folder_name: str
  '''
  # Get the input for the file name and the file folder
  while True:
    file_name = str(input('Please enter the Image name - '))
    folder_name = str(input('Please enter the folder name - '))

    if file_name == '' or folder_name == '':
      print('\033[1mPlease enter a valid input.\033[0m')
      print()
    else:
      break
  print()
  print('\033[1mNote -\033[0m If there will be more than one AOI to export, a number will be prefixed to the file name.')
  print()
  return file_name, folder_name

In [ ]:
def is_valid_date_format(date_string):
  '''
  Checks if the date string is in the format YYYY-MM-DD

  Parameter:
  - date_string (str): date string in the format YYYY-MM-DD

  Returns:
  bool: True if the date string is in the format YYYY/MM/DD, False otherwise
  '''
  try:
    datetime.strptime(date_string, "%Y-%m-%d")
    return True
  except ValueError:
    return False

In [ ]:
def get_start_and_end_date():
  '''
  This function gets the start and end date.

  Returns:
  - start_date: str
  - end_date: str
  '''
  # Loop to get start date as the input.
  while True:
    start_date = str(input('Please enter the start date in following format (YYYY-MM-DD) - ')).strip()
    if is_valid_date_format(start_date):
      break
    else:
      print('\033[1mPlease enter a valid date.\033[0m')
      print()

  print()

  while True:
    end_date = str(input('Please enter the end date in following format (YYYY-MM-DD) - ')).strip()
    if is_valid_date_format(end_date):
      first_date = datetime.strptime(start_date, "%Y-%m-%d")
      second_date = datetime.strptime(end_date, "%Y-%m-%d")
      if first_date > second_date:
        print('\033[1mEnd date cannot be before start date. Please try again\033[0m')
        print()
      else:
        break
    else:
      print('\033[1mPlease enter a valid date.\033[0m')
      print()
  print()
  return start_date, end_date

In [ ]:
def mask_clouds(img):
  '''
  This function masks the clouds.

  Parameters:
  - img: ee.Image

  Returns:
  - ee.Image
  '''
  clouds = ee.Image(img.get('cloud_mask')).select('probability')
  is_not_cloud = clouds.lt(MAX_CLOUD_PROBABILITY)
  return img.updateMask(is_not_cloud)

In [ ]:
def mask_edges(s2_img):
  '''
  This function masks the edges.

  Parameters:
  - s2_img: ee.Image

  Returns:
  - ee.Image
  '''
  return s2_img.updateMask(
      s2_img.select('B8').mask().updateMask(s2_img.select('B9').mask())
  )

In [ ]:
def split_image_collection_by_month(collection):
  '''
  This function splits the image collection by month.

  Parameters:
  - collection: ee.ImageCollection

  Returns:
  - list of ee.ImageCollection
  '''
  return [collection.filter(ee.Filter.calendarRange(i, i, "month")) for i in range(1, 13)]

In [ ]:
def split_collection_img_mean(split_collection_list):
  '''
  This function does the mean on the image collection by month.

  Parameters:
  - split_collection_list: list of ee.ImageCollection

  Returns:
  - list of ee.Image
  '''
  median_list = []

  for img in split_collection_list:
    median_list.append(img.mean())
  return median_list

In [ ]:
def split_img_collection_gap_filled(split_collection_list):
  '''
  This function does the gap filling on the image collection by month.

  Parameters:
  - split_collection_list: list of ee.ImageCollection

  Returns:
  - list of ee.Image
  '''
  gap_filled_list = []

  for i in range(0, len(split_collection_list)):
    if i == 0:
      current_image = split_collection_list[i]
      next_image = split_collection_list[i+1]
      temp_image = ee.ImageCollection([current_image,
                                      next_image]).mean()
    elif i ==  len(split_collection_list) - 1:
      previous_image = split_collection_list[i-1]
      current_image = split_collection_list[i]
      temp_image = ee.ImageCollection([previous_image,
                                      current_image]).mean()
    else:
      previous_image = split_collection_list[i-1]
      current_image = split_collection_list[i]
      next_image = split_collection_list[i+1]
      temp_image = ee.ImageCollection([previous_image,
                                      current_image, next_image]).mean()
    gap_filled_list.append(temp_image)

  return gap_filled_list

In [ ]:
def to_double(image):
  '''
  This function converts the image to double.

  Parameters:
  - image: ee.Image

  Returns:
  - ee.Image
  '''
  return image.float()

In [ ]:
def get_bands_img(img_list, AOI):
  '''
  This function clears the bands and clips the image.

  Parameters:
  - img_list: ee.Image
  - AOI: ee.FeatureCollection

  Returns:
  - ee.Image
  '''
  band_list = ['B4', 'B3', 'B2']
  twelve_month_band_img = img_list.select(band_list).clip(AOI)
  return twelve_month_band_img

In [ ]:
def get_label_image(AOI):
  '''
  This function gets the label image.

  Parameters:
  - AOI: ee.FeatureCollection

  Returns:
  - ee.Image
  '''
  JB_mosaic_name = "projects/wri-datalab/TropicalTreeCover"
  jb_img_collection = ee.ImageCollection(JB_mosaic_name)
  jb_img = jb_img_collection.median().clip(AOI)
  return (jb_img.float())

In [ ]:
def add_labels(img, labels):
  '''
  This function adds the labels to the image.

  Parameters:
  - img: ee.Image
  - labels: ee.Image

  Returns:
  - ee.Image
  '''
  return img.addBands(labels.rename('target'))

### Get the coordinates for export.

You need to paste the coordinates here.

In [ ]:
# The line just below is to hide the block as it can become really big
#@title Training AOI list
training_aoi = [[[[30.331059548460626, -1.2362048774806378],
           [30.331059548460626, -1.3268194046893358],
           [30.43336972912469, -1.3268194046893358],
           [30.43336972912469, -1.2362048774806378]]],
         [[[30.43062314709344, -1.2382643337556833],
           [30.43062314709344, -1.3247600196189961],
           [30.50752744396844, -1.3247600196189961],
           [30.50752744396844, -1.2382643337556833]]],
         [[[30.32899961193719, -1.3261329431895894],
           [30.32899961193719, -1.3810492531243461],
           [30.44435605724969, -1.3810492531243461],
           [30.44435605724969, -1.3261329431895894]]],
         [[[30.440236184202814, -1.3295652487824567],
           [30.440236184202814, -1.3810492531243461],
           [30.50821408947625, -1.3810492531243461],
           [30.50821408947625, -1.3295652487824567]]],
         [[[30.124981980377438, -1.4054212697985782],
           [30.124981980377438, -1.5729061281774923],
           [30.226605515533688, -1.5729061281774923],
           [30.226605515533688, -1.4054212697985782]]],
         [[[30.223858933502438, -1.4040483915017943],
           [30.223858933502438, -1.5729061281774923],
           [30.292523484283688, -1.5729061281774923],
           [30.292523484283688, -1.4040483915017943]]],
         [[[30.411999802643063, -1.4781826450694613],
           [30.411999802643063, -1.5811427533056006],
           [30.536969285064938, -1.5811427533056006],
           [30.536969285064938, -1.4781826450694613]]],
         [[[30.538342576080563, -1.476809810632959],
           [30.538342576080563, -1.5825155209872854],
           [30.646832566314938, -1.5825155209872854],
           [30.646832566314938, -1.476809810632959]]],
         [[[30.413373093658688, -1.5783972152193706],
           [30.413373093658688, -1.6607617525131488],
           [30.541089158111813, -1.6607617525131488],
           [30.541089158111813, -1.5783972152193706]]],
         [[[30.546582322174313, -1.5825155209872854],
           [30.546582322174313, -1.656643605763439],
           [30.649579148346188, -1.656643605763439],
           [30.649579148346188, -1.5825155209872854]]],
         [[[29.585050951422133, -1.451860710181182],
           [29.585050951422133, -1.5658043251798754],
           [29.708647142828383, -1.5658043251798754],
           [29.708647142828383, -1.451860710181182]]],
         [[[29.705900560797133, -1.4477421560065906],
           [29.705900560797133, -1.5644315464997942],
           [29.816450487554945, -1.5644315464997942],
           [29.816450487554945, -1.4477421560065906]]],
         [[[29.585737596929945, -1.5658043251798754],
           [29.585737596929945, -1.6200283558041713],
           [29.737486254156508, -1.6200283558041713],
           [29.737486254156508, -1.5658043251798754]]],
         [[[29.73542631763307, -1.5692362679448861],
           [29.73542631763307, -1.6179692416044098],
           [29.824003588140883, -1.6179692416044098],
           [29.824003588140883, -1.5692362679448861]]],
         [[[29.701247701473505, -1.7615693014122493],
           [29.701247701473505, -1.8658871153161058],
           [29.827590474911005, -1.8658871153161058],
           [29.827590474911005, -1.7615693014122493]]],
         [[[29.830337056942255, -1.7615693014122493],
           [29.830337056942255, -1.87686758391245],
           [29.953933248348505, -1.87686758391245],
           [29.953933248348505, -1.7615693014122493]]],
         [[[29.695754537411005, -1.8603968552846895],
           [29.695754537411005, -1.979806068706419],
           [29.85642958623913, -1.979806068706419],
           [29.85642958623913, -1.8603968552846895]]],
         [[[29.852309713192255, -1.8741224732124158],
           [29.852309713192255, -1.977061123933416],
           [29.96629286748913, -1.977061123933416],
           [29.96629286748913, -1.8741224732124158]]],
         [[[30.04594374639538, -1.7615693014122493],
           [30.04594374639538, -1.8864754373811676],
           [30.203872213192255, -1.8864754373811676],
           [30.203872213192255, -1.7615693014122493]]],
         [[[30.206618795223505, -1.7711777706158833],
           [30.206618795223505, -1.8809852418795672],
           [30.321975240536005, -1.8809852418795672],
           [30.321975240536005, -1.7711777706158833]]],
         [[[30.047317037411005, -1.8796126903026291],
           [30.047317037411005, -1.9633363321006605],
           [30.20524550420788, -1.9633363321006605],
           [30.20524550420788, -1.8796126903026291]]],
         [[[30.206618795223505, -1.8727499162481653],
           [30.206618795223505, -1.9647088163685376],
           [30.35905409795788, -1.9647088163685376],
           [30.35905409795788, -1.8727499162481653]]],
         [[[30.677633553881687, -1.8109532300088387],
           [30.677633553881687, -1.9056604644282054],
           [30.762090951342625, -1.9056604644282054],
           [30.762090951342625, -1.8109532300088387]]],
         [[[30.680380135912937, -1.9070329953782665],
           [30.680380135912937, -1.9852654256252071],
           [30.764837533373875, -1.9852654256252071],
           [30.764837533373875, -1.9070329953782665]]],
         [[[30.587330959048547, -2.0906839454685042],
           [30.587330959048547, -2.212134508171476],
           [30.699940822329797, -2.212134508171476],
           [30.699940822329797, -2.0906839454685042]]],
         [[[30.565358302798547, -2.0200049632653783],
           [30.565358302798547, -2.094114883888746],
           [30.701314113345422, -2.094114883888746],
           [30.701314113345422, -2.0200049632653783]]],
         [[[30.701314113345422, -2.020691181933566],
           [30.701314113345422, -2.133913217391431],
           [30.811864040103234, -2.133913217391431],
           [30.811864040103234, -2.020691181933566]]],
         [[[30.702000758853234, -2.1352855554576053],
           [30.702000758853234, -2.2430102060845436],
           [30.822850368228234, -2.2430102060845436],
           [30.822850368228234, -2.1352855554576053]]],
         [[[30.570851466861047, -2.2087038352319297],
           [30.570851466861047, -2.2889794794884075],
           [30.701314113345422, -2.2889794794884075],
           [30.701314113345422, -2.2087038352319297]]],
         [[[30.522786281314172, -2.0934286968052858],
           [30.522786281314172, -2.210076105359141],
           [30.587330959048547, -2.210076105359141],
           [30.587330959048547, -2.0934286968052858]]],
         [[[30.256093818631832, -2.191694404460653],
           [30.256093818631832, -2.27608760958975],
           [30.359090644803707, -2.27608760958975],
           [30.359090644803707, -2.191694404460653]]],
         [[[30.232061225858395, -2.1512114113128282],
           [30.232061225858395, -2.1910082610969828],
           [30.363897163358395, -2.1910082610969828],
           [30.363897163358395, -2.1512114113128282]]],
         [[[30.21146186062402, -2.190322117418959],
           [30.21146186062402, -2.2815764294014036],
           [30.257467109647457, -2.2815764294014036],
           [30.257467109647457, -2.190322117418959]]],
         [[[30.213521797147457, -2.280204226410549],
           [30.213521797147457, -2.3515570283715834],
           [30.36252387234277, -2.3515570283715834],
           [30.36252387234277, -2.280204226410549]]],
         [[[29.89285834499902, -2.130626425182913],
           [29.89285834499902, -2.230804053342253],
           [29.99722846218652, -2.230804053342253],
           [29.99722846218652, -2.130626425182913]]],
         [[[29.993108589139645, -2.1059240788627425],
           [29.993108589139645, -2.191694404460653],
           [30.084432441678707, -2.191694404460653],
           [30.084432441678707, -2.1059240788627425]]],
         [[[29.99722846218652, -2.1937528326644244],
           [29.99722846218652, -2.285693030514728],
           [30.110524970975582, -2.285693030514728],
           [30.110524970975582, -2.1937528326644244]]],
         [[[29.864705879178707, -2.2287456765358886],
           [29.864705879178707, -2.287751326645699],
           [30.019887763944332, -2.287751326645699],
           [30.019887763944332, -2.2287456765358886]]],
         [[[29.849599678006832, -2.1374881179294567],
           [29.849599678006832, -2.2294318024579187],
           [29.966329414334957, -2.2294318024579187],
           [29.966329414334957, -2.1374881179294567]]],
         [[[29.875692207303707, -2.0922003836127625],
           [29.875692207303707, -2.170423814447053],
           [30.023320991483395, -2.170423814447053],
           [30.023320991483395, -2.0922003836127625]]],
         [[[29.712270576444332, -2.134057275383855],
           [29.712270576444332, -2.2136508255226697],
           [29.820760566678707, -2.2136508255226697],
           [29.820760566678707, -2.134057275383855]]],
         [[[29.710210639920895, -2.2102201560887433],
           [29.710210639920895, -2.2843208314542753],
           [29.822133857694332, -2.2843208314542753],
           [29.822133857694332, -2.2102201560887433]]],
         [[[29.383367378202145, -1.9062327995247645],
           [29.383367378202145, -2.0016210199882853],
           [29.503530342069332, -2.0016210199882853],
           [29.503530342069332, -1.9062327995247645]]],
         [[[29.502157051053707, -1.8897623484708428],
           [29.502157051053707, -2.002307246388325],
           [29.61545355984277, -2.002307246388325],
           [29.61545355984277, -1.8897623484708428]]],
         [[[29.367574531522457, -1.9975036555594705],
           [29.367574531522457, -2.0997484309113243],
           [29.514516670194332, -2.0997484309113243],
           [29.514516670194332, -1.9975036555594705]]],
         [[[29.515889961209957, -2.002307246388325],
           [29.515889961209957, -2.10112079923437],
           [29.625753242459957, -2.10112079923437],
           [29.625753242459957, -2.002307246388325]]],
         [[[29.33191038123047, -2.3058528950393526],
           [29.33191038123047, -2.3799486137447947],
           [29.421174297246093, -2.3799486137447947],
           [29.421174297246093, -2.3058528950393526]]],
         [[[29.27697874060547, -2.2688035809787546],
           [29.27697874060547, -2.3744601770376703],
           [29.374482402714843, -2.3744601770376703],
           [29.374482402714843, -2.2688035809787546]]],
         [[[29.16711545935547, -2.4348317707259866],
           [29.16711545935547, -2.53498773427253],
           [29.26599241248047, -2.53498773427253],
           [29.26599241248047, -2.4348317707259866]]],
         [[[29.26599241248047, -2.440319964309158],
           [29.26599241248047, -2.5363596806569833],
           [29.37310911169922, -2.5363596806569833],
           [29.37310911169922, -2.440319964309158]]],
         [[[29.16436887732422, -2.53498773427253],
           [29.16436887732422, -2.596723869783004],
           [29.275605449589843, -2.596723869783004],
           [29.275605449589843, -2.53498773427253]]],
         [[[29.27972532263672, -2.537731625586219],
           [29.27972532263672, -2.6145581948990455],
           [29.39233518591797, -2.6145581948990455],
           [29.39233518591797, -2.537731625586219]]],
         [[[29.500825176152343, -2.4581564384680608],
           [29.500825176152343, -2.569285955517392],
           [29.63128782263672, -2.569285955517392],
           [29.63128782263672, -2.4581564384680608]]],
         [[[29.632661113652343, -2.462272514101476],
           [29.632661113652343, -2.5761454896056852],
           [29.750764140996093, -2.5761454896056852],
           [29.750764140996093, -2.462272514101476]]],
         [[[29.49121213904297, -2.566542131558285],
           [29.49121213904297, -2.6735468395165047],
           [29.63128782263672, -2.6735468395165047],
           [29.63128782263672, -2.566542131558285]]],
         [[[29.63403440466797, -2.577517391992572],
           [29.63403440466797, -2.691380060390003],
           [29.772736797246093, -2.691380060390003],
           [29.772736797246093, -2.577517391992572]]],
         [[[29.478852519902343, -2.6776622212477204],
           [29.478852519902343, -2.7709404581140764],
           [29.616181621464843, -2.7709404581140764],
           [29.616181621464843, -2.6776622212477204]]],
         [[[29.613435039433593, -2.680405801389615],
           [29.613435039433593, -2.7654537040361373],
           [29.71643186560547, -2.7654537040361373],
           [29.71643186560547, -2.680405801389615]]]]

In [ ]:
while len(training_aoi) <= 0:
  print('This bloack will continue until there are coordinates here.')
  print('You will need to break the cell once you enter the coordinates.')
  print()

print('You have got coordinates to work with.')

You have got coordinates to work with.


In [ ]:
list_training_aoi = []
for i in training_aoi:
  for j in i:
    list_training_aoi.append(ee.Geometry.Polygon(j))

### Exporting the Sentinel Image.

After providing with file name and the folder name, Sentinel images will be exported. The scale of the export will be 10 which is the original resolution of planet image.

You can look the current status of your export task at -
[Export Tasks](https://code.earthengine.google.com/tasks)

In [ ]:
def export_image_collection_to_drive(img, folder_name, file_name, n, AOI):
  '''
  This function exports the image to the drive

  Parameters
  - img: The image needs to be exported
  - folder_name: The name of the folder in which the image needs to be exported
  - file_name: The name of the file
  - n: The number of the image
  - AOI: The area of interest
  '''
  description = f'{n}_{file_name}'
  img = img.clip(AOI)
  task = ee.batch.Export.image.toDrive(**{
      "image": img,
      "description": description,
      "folder": folder_name,
      "scale": 10, #This is the original scale for the Planet Image
      "maxPixels": 1e13,
      "region": AOI,
      'fileFormat': 'GeoTIFF'
  })

  task.start()
  print(f"{n}) Exporting image to drive - {description}")

In [ ]:
# Get the input for the file name and the file folder
while True:
  file_name = str(input('Please enter the file name - '))
  folder_name = str(input('Please enter the folder name - '))

  if file_name == '' or folder_name == '':
    print('\033[1mPlease enter a valid input.\033[0m')
    print()
  else:
    break
print()
print('\033[1mNote -\033[0m If there will be more than one AOI to export, a number will be prefixed to the file name.')
print()

Please enter the file name - example_rwanda_sentinel_export
Please enter the folder name - Example Rwanda Sentinel Export

Note - If there will be more than one AOI to export, a number will be prefixed to the file name.



### Start the export process.

Enter the required input values.


In [ ]:
# get the start and the end date
start_date, end_date = get_start_and_end_date()

Please enter the start date in following format (YYYY-MM-DD) - 2022-01-01

Please enter the end date in following format (YYYY-MM-DD) - 2022-12-31



In [ ]:
# Get the sentinel type
sentinel_type = get_sentinel_type()

Enter the sentinel type: (Sentinel 1: 1, Sentinel 2: 2) - 2


**Please run this block only if needed**

In [ ]:
# This is filtering criteria based on the start and end date
criteria = ee.Filter.And(ee.Filter.date(start_date, end_date))

s2sr = ee.ImageCollection(sentinel_type)
s2Clouds = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
s2Sr = s2sr.filter(criteria).map(mask_edges)
s2Clouds = s2Clouds.filter(criteria)

# Applying the join on s2 and s2_clouds image collection for masking clouds
s2Sr_with_cloud_mask = ee.Join.saveFirst('cloud_mask').apply(**{
    'primary': s2Sr,
    'secondary': s2Clouds,
    'condition': ee.Filter.equals(**{
        'leftField': 'system:index',
        'rightField': 'system:index'
    })
})

# Masking the clouds
s2_cloud_masked = ee.ImageCollection(s2Sr_with_cloud_mask).map(mask_clouds)

# Split the image collection by months
monthly_list = split_image_collection_by_month(s2_cloud_masked)

# Get the mean image for each month
s2_mean_list = split_collection_img_mean(monthly_list)

# Gap fill the image from the previous and next month
s2_gap_filled_list = split_img_collection_gap_filled(s2_mean_list)

# Get the mean of all the images
s2_gap_filled_image = ee.ImageCollection.fromImages(s2_gap_filled_list).mean()

# Export the images
for n, AOI in enumerate(list_training_aoi):
  # Get the John Brandt product as the label.
  labels = get_label_image(AOI)

  # Add the John Brandt band
  output_image = s2_gap_filled_image.addBands(labels.rename('target'))

  img = get_bands_img(output_image, AOI)
  img = img.float()
  try:
    export_image_collection_to_drive(img, folder_name, file_name, n, AOI)
  except:
    print('There was an error in exporting the image.')


0) Exporting image to drive - 0_example_rwanda_sentinel_export
1) Exporting image to drive - 1_example_rwanda_sentinel_export
2) Exporting image to drive - 2_example_rwanda_sentinel_export
3) Exporting image to drive - 3_example_rwanda_sentinel_export
4) Exporting image to drive - 4_example_rwanda_sentinel_export
5) Exporting image to drive - 5_example_rwanda_sentinel_export
6) Exporting image to drive - 6_example_rwanda_sentinel_export
7) Exporting image to drive - 7_example_rwanda_sentinel_export
8) Exporting image to drive - 8_example_rwanda_sentinel_export
9) Exporting image to drive - 9_example_rwanda_sentinel_export
10) Exporting image to drive - 10_example_rwanda_sentinel_export
11) Exporting image to drive - 11_example_rwanda_sentinel_export
12) Exporting image to drive - 12_example_rwanda_sentinel_export
13) Exporting image to drive - 13_example_rwanda_sentinel_export
14) Exporting image to drive - 14_example_rwanda_sentinel_export
15) Exporting image to drive - 15_example_rwa